In [ ]:
from langchain_openai import ChatOpenAI
from langchain.tools import BaseTool
from langchain.agents import initialize_agent, AgentType
from pydantic import BaseModel, Field
from typing import Any, Type
from langchain_community.tools import WikipediaQueryRun, GoogleSearchRun
from langchain_community.utilities import WikipediaAPIWrapper, GoogleSearchAPIWrapper
from langchain.schema import SystemMessage
from langchain.document_loaders import WebBaseLoader

TA's 힌트
다양한 커스텀 도구를 만들고 이 도구들을 에이전트에게 전달하여 리서치 AI 에이전트를 구현해야 하는 챌린지입니다.

먼저, 위키피디아에서 검색을 수행하는 도구를 만들어야 합니다. 랭체인에서 제공하는 WikipediaQueryRun를 사용하면 간단하게 구현할 수 있습니다. (LangChain - Wikipedia 를 참고하세요.)

다음은 DuckDuckGo에서 검색을 수행하는 도구를 만들어야 합니다. 이 또한 랭체인에서 제공하는 DuckDuckGoSearchRun를 사용하면 덕덕고의 검색 결과를 얻을 수 있습니다. (LangChain - DuckDuckGo Search 를 참고하세요.)

DuckDuckGo를 통해 얻은 검색 결과에서 웹사이트들의 콘텐츠를 추출하는 도구가 필요합니다. 다양한 방법이 있지만, 그중에서도 랭체인이 제공하는 WebBaseLoader를 이용하면 웹사이트의 콘텐츠를 쉽게 스크랩할 수 있습니다. (LangChain - WebBaseLoader 를 참고하세요.)

마지막으로, 리서치 결과를 txt 파일에 저장하는 도구가 필요합니다. 파이썬에서 기본으로 제공하는 파일 쓰기 기능을 이용하여 구현하세요. (python - reading-and-writing-files)


In [2]:
llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o-mini",
)

In [3]:
class WikipediaSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for Example query: Search for Apple Company."
    )


class WikipediaSearchTool(BaseTool):
    name: str = "WikipediaSearchTool"
    description: str = (
        """
        Wikipedia에서 검색을 할 때 사용하는 툴입니다.
        이 툴은 query를 입력받습니다.
        query의 예: "Tesla Motors" 
        """
    )
    args_schema: Type[WikipediaSearchToolArgsSchema] = WikipediaSearchToolArgsSchema

    def _run(self, query):
        wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        return wikipedia.run(query)

In [4]:
class SearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search websites about Example query: Search websites about Apple Company."
    )


class SearchTool(BaseTool):
    name: str = "SearchTool"
    description: str = (
        """
        Google 검색 엔진을 사용해서 웹 검색을 할 때 이 툴을 사용합니다.
        query의 예: "weather forecast"
        """
    )
    args_schema: Type[SearchToolArgsSchema] = SearchToolArgsSchema

    def _run(self, query):
        google = GoogleSearchAPIWrapper()
        return google.run(query)

In [23]:
class SaveToolArgsSchema(BaseModel):
    filename: str = Field(description="Filename to save the search results.")
    contents: str = Field(description="the search results.")


class SaveTool(BaseTool):
    name: str = "SaveTool"
    description: str = "Use this tool to save search results in text file."
    args_schema: Type[SaveToolArgsSchema] = SaveToolArgsSchema

    def _run(self, filename, contents):
        # print(filename, contents)
        f = open(filename, "w", encoding="utf-8")
        f.write(contents)
        f.close()

In [31]:
class WebScrapingToolArgsSchema(BaseModel):
    url: str = Field(description="The URL of the website you want to scrape")


class WebScrapingTool(BaseTool):
    name: str = "WebScrapingTool"
    description: str = (
        "Google에서 wesite link를 찾으면 이 툴을 사용해서 링크의 내용을 가져옵니다."
    )
    args_schema: Type[WebScrapingToolArgsSchema] = WebScrapingToolArgsSchema

    def _run(self, url):
        loader = WebBaseLoader([url])
        docs = loader.load()
        text = "\n\n".join([doc.page_content for doc in docs])
        return text

In [32]:
system_message = SystemMessage(
    content="""
    당신은 검색 전문가입니다.
    
    당신은 제공받은 query를 Wikipedia 또는 Google에서 정확한 정보를 수집합니다.
    
    Wikipedia에서 찾은 결과와 Google에서 검색한 정보와 관련 웹사이트에서 찾은 정보를 결합합니다.
    결합한 내용이 잘 구성되고 자세한지 확인하고 사용된 모든 출처에 대한 링크(URL)가 있는 인용문을 포함합니다.
    
    최종 결과물은 txt파일로 저정해야합니다. 
    """
)

In [34]:
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        SearchTool(),
        WebScrapingTool(),
        SaveTool(),
    ],
    agent_kwargs={
        "system_message": system_message,
    },
)

prompt = "Research about the XZ backdoor"

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: In February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been given the Common Vulnerabilities and Exposures number CVE-2024-3094 and has been assigned a CVSS score of 10.0, the highest possible score.
While xz is commonly present in most Linux distributions, at the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions. The backdoor was discovered by the software developer Andres Freund, who announced his findings on 29 March 2024.



Page: XZ Utils
Summary: XZ Utils 

{'input': 'Research about the XZ backdoor',
 'output': 'The research on the XZ backdoor has been compiled and saved successfully. Here are the details:\n\n### XZ Backdoor Research\n\n#### Overview\nIn February 2024, a malicious backdoor was introduced to the Linux build of the xz utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". This backdoor allows an attacker who possesses a specific Ed448 private key to execute remote code on the affected Linux system. The issue has been assigned the Common Vulnerabilities and Exposures (CVE) number CVE-2024-3094 and has been given a CVSS score of 10.0, indicating the highest severity level.\n\n#### Discovery\nThe backdoor was discovered by software developer Andres Freund, who announced his findings on March 29, 2024. At the time of discovery, the backdoored version had not been widely deployed to production systems but was present in development versions of major Linux distributions.\n\n#### Tec